In [1]:
!dir

 Volume in drive D is Samip
 Volume Serial Number is 3A4B-753D

 Directory of d:\TessTutorial

08/29/2024  12:52 PM    <DIR>          .
08/29/2024  10:56 AM           642,324 NotoSansDevanagari-VariableFont_wdth,wght.ttf
08/29/2024  11:20 AM    <DIR>          tesseract_tutorial
08/29/2024  01:03 PM             3,989 train.ipynb
08/29/2024  12:34 PM               145 train.txt
               3 File(s)        646,458 bytes
               2 Dir(s)   2,250,174,464 bytes free


In [20]:
#switch to  tesstrain
%cd tesstrain
!dir

d:\TessTutorial\tesseract_tutorial\tesstrain
 Volume in drive D is Samip
 Volume Serial Number is 3A4B-753D

 Directory of d:\TessTutorial\tesseract_tutorial\tesstrain

08/29/2024  12:57 PM    <DIR>          .
08/29/2024  11:20 AM    <DIR>          ..
08/29/2024  11:20 AM    <DIR>          .github
08/29/2024  11:20 AM                28 .gitignore
08/29/2024  11:20 AM               264 .pylintrc
08/29/2024  11:20 AM             1,634 count_chars.py
08/29/2024  01:07 PM    <DIR>          data
08/29/2024  11:20 AM             1,068 generate_eval_train.py
08/29/2024  11:20 AM             1,095 generate_gt_from_box.py
08/29/2024  11:20 AM             1,527 generate_line_box.py
08/29/2024  11:20 AM             2,336 generate_line_syllable_box.py
08/29/2024  11:20 AM             1,316 generate_wordstr_box.py
08/29/2024  11:20 AM            10,349 LICENSE
08/29/2024  11:20 AM            17,854 Makefile
08/29/2024  11:20 AM             1,385 normalize.py
08/29/2024  11:20 AM         5,524,523 o

## SAMPLE TRAINING TEST

Training is done through bash command

In [37]:
import os
import subprocess

# Step 1: Set environment variables
os.environ['TESSDATA_PREFIX'] = r'..\tesseract\tessdata'
os.environ['GROUND_TRUTH_DIR'] = r'..\data\nep-ocr-ground-truth'

# Step 2: Prepend the desired directory to the PATH environment variable
git_bin_path = r'C:\Program Files\Git\usr\bin'
os.environ['PATH'] = git_bin_path + os.pathsep + os.environ['PATH']

# Step 3: Change the working directory to where the ground truth files are located
os.chdir(r'..\data\nep-ocr-ground-truth')

# Debug: Print the current working directory and list of files
print("Current working directory:", os.getcwd())
print("Files in the current directory:", os.listdir())

# Step 4: Verify the presence of ground truth files
ground_truth_files = [f for f in os.listdir() if f.endswith('.gt.txt')]
if not ground_truth_files:
    raise FileNotFoundError("No ground truth files (*.gt.txt) found in the directory.")

# Step 5: Change the working directory to the tesstrain folder
os.chdir(r'..\..\tesstrain')

# Step 6: Define the command
command = [
    'make', 'training',
    'MODEL_NAME=nep-ocr',
    'START_MODEL=nep',
    'TESSDATA=../tesseract/tessdata',
    'MAX_ITERATIONS=100',
    'LEARNING_RATE=0.002',
    'LANG_TYPE=Indic'
]

# Step 7: Run the command and print output in real-time
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8')

# Step 8: Print the output line by line
for line in process.stdout:
    print(line, end='')

# Step 9: Wait for the process to complete and get the return code
process.wait()
print(f"Process finished with return code {process.returncode}")

Current working directory: d:\TessTutorial\tesseract_tutorial\data\nep-ocr-ground-truth
Files in the current directory: ['1.gt.txt', '1.png', '10.gt.txt', '10.png', '11.gt.txt', '11.png', '12.gt.txt', '12.png', '13.gt.txt', '13.png', '14.gt.txt', '14.png', '15.gt.txt', '15.png', '16.gt.txt', '16.png', '17.gt.txt', '17.png', '18.gt.txt', '18.png', '19.gt.txt', '19.png', '2.gt.txt', '2.png', '20.gt.txt', '20.png', '21.gt.txt', '21.png', '22.gt.txt', '22.png', '23.gt.txt', '23.png', '24.gt.txt', '24.png', '25.gt.txt', '25.png', '26.gt.txt', '26.png', '27.gt.txt', '27.png', '28.gt.txt', '28.png', '29.gt.txt', '29.png', '3.gt.txt', '3.png', '30.gt.txt', '30.png', '31.gt.txt', '31.png', '32.gt.txt', '32.png', '33.gt.txt', '33.png', '34.gt.txt', '34.png', '35.gt.txt', '35.png', '36.gt.txt', '36.png', '37.gt.txt', '37.png', '38.gt.txt', '38.png', '39.gt.txt', '39.png', '4.gt.txt', '4.png', '40.gt.txt', '40.png', '41.gt.txt', '41.png', '42.gt.txt', '42.png', '43.gt.txt', '43.png', '44.gt.txt', 

In [33]:
import subprocess

# Define the command and its arguments
command = [
    'lstmtraining',
    '--stop_training',
    '--continue_from', 'data/nep-ocr/checkpoints/nep-ocr_checkpoint',
    '--traineddata', 'data/nep-ocr/nep-ocr.traineddata',
    '--model_output', 'data/nep-ocr-new.traineddata'
]

# Run the command and print output in real-time
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8')

# Print the output line by line
for line in process.stdout:
    print(line, end='')

# Wait for the process to complete and get the return code
process.wait()
print(f"Process finished with return code {process.returncode}")

Loaded file data/nep-ocr/checkpoints/nep-ocr_checkpoint, unpacking...
Process finished with return code 0
